In [15]:
import os
import requests
import time
import boto3
import pandas as pd
import jupyter_black
from io import BytesIO
from io import StringIO
from tqdm.notebook import tqdm
import altair as alt

In [16]:
jupyter_black.load()
pd.options.display.max_columns = 200
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [17]:
# Headers for requests
headers = {
    "sec-ch-ua": '"Chromium";v="124", "Google Chrome";v="124", "Not-A.Brand";v="99"',
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36",
    "sec-ch-ua-platform": '"macOS"',
}

In [18]:
batter_list = requests.get(
    "https://bdfed.stitch.mlbinfra.com/bdfed/stats/player?&env=prod&season=2024&sportId=1&stats=season&group=hitting&gameType=R&offset=0&sortStat=homeRuns&order=desc&teamId=119",
    headers=headers,
)

In [19]:
batters = pd.DataFrame(batter_list.json()["stats"])

In [20]:
players_ids = batters["playerId"].to_list()

In [ ]:
at_bats_list = []

for player in tqdm(players_ids):
    params = {
        "warehouse": "true",
        "hfPT": "",
        "hfAB": "",
        "hfGT": "",
        "hfPR": "",
        "hfZ": "",
        "hfStadium": "",
        "hfBBL": "",
        "hfNewZones": "",
        "hfPull": "",
        "hfC": "",
        "hfSea": "2024|",
        "hfSit": "",
        "player_type": "batter",
        "batters_lookup[]": player,
        "hfFlag": "",
        "metric_1": "",
        "group_by": "name",
        "min_pitches": "0",
        "min_results": "0",
        "min_pas": "0",
        "sort_col": "pitches",
        "player_event_sort": "",
        "sort_order": "desc",
        "type": "details",
        "player_id": player,
        "minors": "false",
    }

    json_data = requests.get(
        "https://baseballsavant.mlb.com/player/spray", params=params, headers=headers
    ).json()

    at_bats_src = pd.DataFrame(json_data)
    at_bats_list.append(at_bats_src)
    time.sleep(3)

In [ ]:
src = pd.concat(at_bats_list).reset_index(drop=True)

In [30]:
src.iloc[0]

player_id                                                                             660271
batter_name                                                                   Ohtani, Shohei
pitcher_name                                                                   Megill, Tylor
batter                                                                                660271
pitcher                                                                               656731
venue                                                                             Citi Field
team_batting                                                                             LAD
team_pitching                                                                            NYM
pitch_type                                                                                FC
game_date                                                                         2024-05-28
release_speed                                                         

In [ ]:
hit_into_play_df = (
    src[
        [
            "play_id",
            "player_id",
            "game_date",
            "home_team",
            "away_team",
            "team_pitching",
            "inning",
            "inning_topbot",
            "player_id",
            "batter_name",
            "outs_when_up",
            "event_type",
            "events",
            "bb_type",
            "hit_distance_sc",
            "is_hit_into_play_basehit",
            "des",
        ]
    ]
    .sort_values(["game_date", "inning"])
    .reset_index(drop=True)
)

In [68]:
hit_into_play_df["hit_play_num"] = hit_into_play_df.groupby(
    ["player_id", "batter_name"]
)["play_id"].cumcount()

In [71]:
shoehei = hit_into_play_df.query('batter_name == "Ohtani, Shohei"')

In [82]:
hit_plays = hit_into_play_df[
    [
        "hit_play_num",
        "batter_name",
        "game_date",
        "inning",
        "team_pitching",
        "events",
        "bb_type",
        "is_hit_into_play_basehit",
    ]
].copy()

In [83]:
hit_plays

,hit_play_num,batter_name,game_date,inning,team_pitching,events,bb_type,is_hit_into_play_basehit
0,0,"Hernández, Teoscar",2024-02-22,1,SD,double,fly_ball,1
1,0,"Betts, Mookie",2024-02-22,1,SD,field_out,line_drive,0
2,0,"Pages, Andy",2024-02-22,1,SD,single,line_drive,1
3,0,"Taylor, Chris",2024-02-22,1,SD,single,line_drive,1
4,1,"Taylor, Chris",2024-02-22,1,SD,field_out,line_drive,0
...,...,...,...,...,...,...,...,...
1886,62,"Taylor, Chris",2024-05-28,9,NYM,single,ground_ball,1
1887,167,"Hernández, Teoscar",2024-05-28,10,NYM,single,ground_ball,1
1888,234,"Betts, Mookie",2024-05-28,10,NYM,single,ground_ball,1
1889,208,"Freeman, Freddie",2024-05-28,10,NYM,home_run,fly_ball,1


In [132]:
stars = ["Betts, Mookie", "Freeman, Freddie", "Ohtani, Shohei", "Smith, Will"]

In [148]:
alt.Chart(hit_plays.query(f"batter_name.isin({stars})")).mark_tick(
    size=50, thickness=1
).encode(
    x=alt.X("hit_play_num:Q", axis=alt.Axis(grid=False)),
    y=alt.Y("batter_name:O", axis=alt.Axis(grid=False)),
    color=alt.condition(
        "datum.events == 'double'",
        alt.value("#005a9c"),
        alt.value("#ccc"),
    ),
    tooltip=["game_date:T", "events", "inning"],
).properties(
    width=600, height=300
).configure_view(
    strokeWidth=0
)

alt.Chart(...)

In [84]:
Last 50 at bats trend or something...

SyntaxError: invalid syntax (238525314.py, line 1)

In [ ]:
Shoehei specific...